<a href="https://colab.research.google.com/github/illusoryTwin/InnoML/blob/main/test2/Untitled41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values of the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [6]:
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import imgaug.augmenters as iaa

from skimage.util import random_noise
import numpy as np

# Define the desired height and width for resizing
desired_height = 32
desired_width = 32

# datagen = ImageDataGenerator(
#     rescale=1./255,            # Rescale pixel values to [0,1]
#     rotation_range=20,         # Rotate images by up to 20 degrees
#     width_shift_range=0.1,     # Shift images horizontally by up to 10% of width
#     height_shift_range=0.1,    # Shift images vertically by up to 10% of height
#     brightness_range=[0.8, 1.2],  # Vary brightness between 0.8 and 1.2
#     horizontal_flip=True,      # Flip images horizontally
#     vertical_flip=True,        # Flip images vertically
#     channel_shift_range=100,   # Shift color channels by up to 100
# )

datagen = ImageDataGenerator(
    shear_range=0.1,  # Shear transformation
    rotation_range=15,  # Rotation
    # width_shift_range=0.1,  # Width shift
    # height_shift_range=0.1,  # Height shift
    zoom_range=0.1,  # Zoom
    # channel_shift_range=0.1,  # Color filtering
    preprocessing_function=lambda x: tf.image.resize(x, [desired_height, desired_width]),

)


# Define a function to add random colorful pixels (noise) to the images
def add_colorful_noise(image):
    # Generate random noise with the same shape as the input image
    noise = np.random.randint(0, 256, image.shape, dtype=np.uint8)
    # Add the random noise to the input image
    noisy_image = cv2.add(image, noise)
    return noisy_image

# Define data augmentation parameters with resizing and noise
datagen = ImageDataGenerator(
    shear_range=0.1,             # Shear transformation
    rotation_range=15,           # Rotation
    zoom_range=0.1,              # Zoom
    preprocessing_function=lambda x: add_colorful_noise(x),  # Add colorful noise
    # Add other augmentation parameters as needed
)


# def apply_augmentation(image):
#     # Resize the image
#     resized_image = cv2.resize(image, (desired_height, desired_width))
#     # Add glitter effect
#     glittered_image = add_glitter(resized_image)
#     return glittered_image

# # Define data augmentation parameters
# datagen = ImageDataGenerator(
#     shear_range=0.1,  # Shear transformation
#     zoom_range=0.1,  # Zoom
#     preprocessing_function=apply_augmentation  # Apply resizing and glitter effect
# )
# def add_glitter(image):
#     # Randomly generate glitter spots
#     num_glitters = np.random.randint(50, 200)  # Number of glitter spots
#     for _ in range(num_glitters):
#         # Generate random position and radius for glitter spot
#         x = np.random.randint(0, image.shape[1])
#         y = np.random.randint(0, image.shape[0])
#         radius = np.random.randint(1, 4)
#         # Generate random color for glitter spot
#         color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))
#         # Draw glitter spot with random color
#         cv2.circle(image, (x, y), radius, color, -1)
#     return image

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)


# Train the model with data augmentation
history = model.fit(datagen.flow(x_train, y_train, batch_size=256),
                    epochs=100,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping, reduce_lr])



# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')


Epoch 1/100
196/196 [==============================] - 102s 498ms/step - loss: 1.6979 - accuracy: 0.4472 - val_loss: 3.5312 - val_accuracy: 0.1056 - lr: 0.0010
Epoch 2/100
196/196 [==============================] - 93s 474ms/step - loss: 1.1206 - accuracy: 0.6128 - val_loss: 2.4077 - val_accuracy: 0.2809 - lr: 0.0010
Epoch 3/100
196/196 [==============================] - 93s 473ms/step - loss: 0.8923 - accuracy: 0.6885 - val_loss: 0.9630 - val_accuracy: 0.6649 - lr: 0.0010
Epoch 4/100
196/196 [==============================] - 93s 472ms/step - loss: 0.7545 - accuracy: 0.7349 - val_loss: 0.8452 - val_accuracy: 0.7180 - lr: 0.0010
Epoch 5/100
196/196 [==============================] - 92s 470ms/step - loss: 0.6760 - accuracy: 0.7650 - val_loss: 0.7477 - val_accuracy: 0.7635 - lr: 0.0010
Epoch 6/100
196/196 [==============================] - 93s 476ms/step - loss: 0.6037 - accuracy: 0.7883 - val_loss: 0.7007 - val_accuracy: 0.7771 - lr: 0.0010
Epoch 7/100
196/196 [========================

In [ ]:

# Load test images and labels
test_images = np.load("task_2_test_images.npy")
test_labels = np.load("task_2_test_labels.npy")

# Normalize pixel values of test images
test_images = test_images.astype('float32') / 255.0

# Convert test labels to one-hot encoded format (if needed)
test_labels = to_categorical(test_labels, num_classes=10)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')


# Transfer learning

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values of the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



170498071/170498071 [==============================] - 13s 0us/step


In [4]:
# Load the pre-trained Xception model without the top classification layer
# Load the pre-trained Xception model without the top classification layer
base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))

# Now you can build your model on top of the Xception base model as before
# Add your additional layers, compile the model, and define callbacks

# Freeze the base model layers
base_model.trainable = False

# Create new model on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)


83683744/83683744 [==============================] - 5s 0us/step


In [5]:

# Train the model
history = model.fit(x_train, y_train, batch_size=256, epochs=100,
                    validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
d